# Deep Learning for Business Applications course

## TOPIC 4: Object detection problem. Mask R-CNN demo

### 1. Libraries

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms import v2
from PIL import Image

# check if GPU available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

### 2. Get the model

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

Where is my model?

In [ ]:
!ls -la ~/.cache

In [ ]:
!ls -la ~/.cache/torch/hub/checkpoints

__NOTE:__ keep in mind disk space required for models downloads!

### 3. Model inference utils

In [ ]:
model.eval()

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
print(
    'total COCO classes:',
    len(COCO_INSTANCE_CATEGORY_NAMES)
)

In [ ]:
def get_prediction(model, img_path, threshold):
    """
    Extracts masks, classes, bounding boxes
    from model

    """
    img = Image.open(img_path)
    transform = v2.Compose([v2.ToTensor()])
    img = transform(img)
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1]
    masks = (pred[0]['masks'] > .5).squeeze().detach().cpu().numpy()
    pred_class = [
        COCO_INSTANCE_CATEGORY_NAMES[i] 
        for i in list(pred[0]['labels'].numpy())
    ]
    pred_boxes = [
        [(i[0], i[1]), (i[2], i[3])] 
        for i in list(pred[0]['boxes'].detach().numpy())
    ]
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class

In [ ]:
def random_colour_masks(image):
    """
    Produces the colored masks with random color

    """
    colours = [
      [0, 255, 0],
      [0, 0, 255],
      [255, 0, 0],
      [0, 255, 255],
      [255, 255, 0],
      [255, 0, 255],
      [80, 70, 180],
      [250, 80, 190],
      [245, 145, 50],
      [70, 150, 250],
      [50, 190, 190]
    ]
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    r[image == 1], g[image == 1], b[image == 1] = colours[random.randrange(0, 10)]
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask

In [ ]:
def instance_segmentation_api(model, img_path,
                              threshold=.75, rect_th=2,
                              text_size=1, text_th=2):
    """
    Plots the final result for image

    """
    masks, boxes, pred_cls = get_prediction(model, img_path, threshold)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for i in range(len(masks)):
        rgb_mask = random_colour_masks(masks[i])
        img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
        cv2.rectangle(
            img,
            [int(x) for x in boxes[i][0]],
            [int(x) for x in boxes[i][1]],
            color=(0, 255, 0),
            thickness=rect_th
        )
        cv2.putText(
            img,
            pred_cls[i],
            [int(x) for x in boxes[i][0]],
            cv2.FONT_HERSHEY_SIMPLEX,
            text_size,
            (0, 255, 0),
            thickness=text_th
        )
    plt.figure(figsize=(20, 30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

### 4. Model inference examples

Let's use this excellent [comedy film](https://en.wikipedia.org/wiki/Attack_of_the_Killer_Tomatoes) for our tests.

In [ ]:
DATA_PATH = '/home/jovyan/__DATA/DLBA_F24/topic_04'

In [ ]:
img_path = f'{DATA_PATH}/tomato1.jpg'
instance_segmentation_api(model, img_path)

In [ ]:
masks, boxes, pred_cls = get_prediction(model, img_path, threshold=.7)

In [ ]:
pred_cls[0]

In [ ]:
masks[0]

In [ ]:
masks[0].shape

In [ ]:
num = 0
plt.figure(figsize=(16, 16))
plt.imshow(masks[num])
plt.title(pred_cls[num])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
img_path = f'{DATA_PATH}/tomato2.jpg'
instance_segmentation_api(model, img_path)

More samples for [another comedy](https://en.wikipedia.org/wiki/Ace_Ventura:_Pet_Detective).

In [ ]:
img_path = f'{DATA_PATH}/ace.jpg'
instance_segmentation_api(model, img_path, text_size=2)

...and finally:

In [ ]:
img_path = f'{DATA_PATH}/foods.jpg'
instance_segmentation_api(model, img_path, threshold=.9, text_size=1)